In [3]:
import pandas as pd
import numpy as np

In [4]:
from yahooquery import Ticker
def get_data(ticker, start = 0, end = 0, period = -1):
    stock = Ticker(ticker)
    if period == 'max':
        df = stock.history(period='max')
        
    else:
        df = stock.history(start=start, end=end)
    
    df = df.xs(ticker)
    return df

In [8]:
df = get_data('MGLU3.SA', start='2021-01-03', end='2021-07-31')

# RSI

In [9]:
def rsi(df, periods = 14, ema = True):
    """
    Returns a pd.Series with the relative strength index.
    """
    close_delta = df['close'].diff()

    # Make two series: one for lower closes and one for higher closes
    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)
    
    if ema == True:
        # Use exponential moving average
        ma_up = up.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
        ma_down = down.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
    else:
        # Use simple moving average
        ma_up = up.rolling(window = periods, adjust=False).mean()
        ma_down = down.rolling(window = periods, adjust=False).mean()
        
    rsi = ma_up / ma_down
    rsi = 100 - (100/(1 + rsi))
    return rsi

In [11]:
df['RSI'] = rsi(df)

# SMA 

In [12]:
df['SMA1'] = df['close'].rolling(window=42).mean()

In [13]:
df['SMA2'] = df['close'].rolling(window=252).mean()

In [14]:
df

,open,close,high,volume,low,adjclose,dividends,splits,RSI,SMA1,SMA2
date,,,,,,,,,,,
2018-01-03,2.468750,2.446875,2.542500,41385600.0,2.422500,2.360765,0.0,0.0,NaN,NaN,NaN
2018-01-04,2.500000,2.505625,2.516250,47936000.0,2.478125,2.417448,0.0,0.0,NaN,NaN,NaN
2018-01-05,2.514062,2.456250,2.528437,30684800.0,2.448437,2.369811,0.0,0.0,NaN,NaN,NaN
2018-01-08,2.453125,2.456562,2.503437,28796800.0,2.448437,2.370112,0.0,0.0,NaN,NaN,NaN
2018-01-09,2.476562,2.397500,2.487812,31024000.0,2.384375,2.313128,0.0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2021-07-26,22.730000,22.040001,22.760000,17731100.0,22.020000,22.040001,0.0,0.0,49.206028,21.431429,22.696954
2021-07-27,22.080000,21.450001,22.160000,26917800.0,21.100000,21.450001,0.0,0.0,44.224166,21.473810,22.702530
2021-07-28,21.590000,21.530001,21.780001,17736600.0,21.219999,21.530001,0.0,0.0,45.036749,21.520000,22.706617


# VAR

In [19]:
def var_historic(df, level=1):
    """
    Takes in a series of returns (r), and the percentage level
(level)
    Returns the historic Value at Risk at a specified level
    i.e. returns the number such that "level" percent of the returns
    fall below that number, and the (100-level) percent are above
    """
    r = df['close'].diff()

    r = r.dropna()
    if isinstance(r, pd.DataFrame):
        return r.aggregate(var_historic, level=level)
    elif isinstance(r, pd.Series):
        
        return -np.percentile(r, level)
    else:
        raise TypeError("Expected r to be a Series or DataFrame")

In [20]:
var_historic(df, level=1)

1.5160003662109376